In [1]:
%load_ext autoreload
%autoreload 2
%run A_simple_net.ipynb

Detected PyNN version 0.9.4 and Neo version 0.6.1
2020-12-07T09:15:18+00:00

CPython 3.6.9
IPython 7.15.0

numpy 1.19.0
matplotlib 3.2.2
pyNN 0.9.4
neo 0.6.1
nixio 1.4.9
pandas 1.1.4

compiler   : GCC 8.4.0
system     : Linux
release    : 4.4.0-87-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 32
interpreter: 64bit
host name  : 96c3276e2566
Git hash   : 6673a6ed412b75cef571cd9236f1cdc419ae601e
Git repo   : https://github.com/SpikeAI/2020-11_brainhack_Project7
Git branch : main


In [2]:
%mkdir -p outputs

In [3]:
%mkdir -p Results/20201207/

# scanning over single parameters

## initializing parameters

In [4]:
tag = '2020-12-04_scan_' + simulator

In [5]:
buildCPUTime, simCPUTime, writeCPUTime = run(verbose=True)

2020-12-07 09:15:19 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2020-12-07 09:15:19 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2020-12-07 09:15:19 WARNING: /home/spinnaker/2020-11_brainhack_Project7/benchmark/reports has 4 old reports that have not been closed
2020-12-07 09:15:19 WARNING: /home/spinnaker/2020-11_brainhack_Project7/benchmark/application_generated_data_files has 4 old reports that have not been closed
2020-12-07 09:15:19 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2020-12-07 09:15:19 INFO: Setting time scale factor to 10.
2020-12-07 09:15:19 INFO: Setting machine time step to 100 micro-seconds.
2020-12-07 09:15:19 WARNING: Extra params {'threads': 1, 'filename': 'va.xml', 'label': 'VA'} have been applied to the setup command which we do not consider
2020-12-07 09:15:19 INFO: Simulating for 10000 0.1ms timesteps using a hardware timestep of 10us
2020-12-07 09:15:19 INFO: Starting execution process


0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...
0 Running simulation...


2020-12-07 09:15:23 INFO: Time 0:00:03.273703 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-07 09:15:31 INFO: Time 0:00:08.890400 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-07 09:15:37 INFO: Time 0:00:05.374019 taken by PartitionAndPlacePartitioner
Created spalloc job 5541646
2020-12-07 09:15:37 INFO: Created spalloc job 5541646
Waiting for board power commands to complete.
2020-12-07 09:15:37 INFO: Waiting for board power commands to complete.
2020-12-07 09:15:42 INFO: Time 0:00:05.050148 taken by SpallocAllocator
2020-12-07 09:15:42 INFO: Creating transceiver for 10.11.195.65
2020-12-07 09:15:42 INFO: Working out if machine is booted
2020-12-07 09:15:46 INFO: Attempting to

0 Writing data to file...



Getting v for All Cells
|0%                          50%                         100%|


--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Number of Neurons      : 1000
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Excitatory rate        : 1.37 Hz
Inhibitory rate        : 1.285 Hz
Build time             : 5.7838 ms
Simulation time        : 65769.8 ms
Writing time           : 243.355 ms


## Running Model and Loading Results

In [6]:
import os
import pandas as pd
verbose = True
def scan(variable, values, tag=tag, verbose=verbose):
    filename = 'outputs/{tag}_{variable}.json'.format(tag=tag, variable=variable)
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=[variable, 'buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, value in enumerate(values):
            if verbose: print('{variable=}---{value=}'.format(value=value, variable=variable))
            args = {}
            args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= {variable: value, 'buildCPUTime':buildCPUTime, 'simCPUTime':simCPUTime, 'writeCPUTime':writeCPUTime}
        df.to_json(filename)
    return df

In [7]:
import matplotlib.pyplot as plt

# https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression
def plot(df, do_intercept=True):
    variable = df.columns[0]
    fig, ax = plt.subplots(figsize=(8,8))
    
    for regressor, color in zip(['buildCPUTime', 'simCPUTime', 'writeCPUTime'],
                                ['r', 'g', 'b']):

        import patsy
        if do_intercept:
            y, X = patsy.dmatrices('{regressor} ~ {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')
        else:
            y, X = patsy.dmatrices('{regressor} ~ 0 + {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')

        import statsmodels.api as sm
        fit = sm.OLS(y, X).fit()
        print(fit.summary())

        if do_intercept: intercept = fit.params.Intercept
        slope = fit.params[variable]

        values = np.array(df[variable])
        if do_intercept:
            ax.plot(values, intercept + slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable} + {intercept:.3f}")
        else:
            ax.plot(values, slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable}")
        df.plot(x=variable, y=regressor, c=color, lw=1, ls='-.', ax=ax, legend=False)

    ax.set_xlim(0)
    ax.set_ylabel('Time (ms)')
    ax.set_ylim(0)
    ax.legend()
    return fig, ax

## plot as a function of neuron numbers

In [8]:
import numpy as np
N_pops = np.array([int(k)**2 for k in np.linspace(8**2, 45**2, 14, endpoint=True)**.5])
N_pops

array([  64,  196,  361,  484,  625,  784,  961, 1089, 1225, 1369, 1521,
       1681, 1849, 2025])

In [9]:
results_pop = scan('N_pop', N_pops)

Loaded file


In [10]:
fig, ax = plot(results_pop)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

                            OLS Regression Results                            
Dep. Variable:           buildCPUTime   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.080
Method:                 Least Squares   F-statistic:                   0.03301
Date:                Mon, 07 Dec 2020   Prob (F-statistic):              0.859
Time:                        09:16:27   Log-Likelihood:                -11.424
No. Observations:                  14   AIC:                             26.85
Df Residuals:                      12   BIC:                             28.13
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.7428      0.310     18.536      0.0

/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


In [11]:
fig.savefig(tag + '_N_pop.png')

## plot as a function of simtime

In [12]:
simtimes = np.linspace(500, 3000, 15, endpoint=True)
simtimes

array([ 500.        ,  678.57142857,  857.14285714, 1035.71428571,
       1214.28571429, 1392.85714286, 1571.42857143, 1750.        ,
       1928.57142857, 2107.14285714, 2285.71428571, 2464.28571429,
       2642.85714286, 2821.42857143, 3000.        ])

In [13]:
results_simtime = scan('simtime', simtimes)

Loaded file


In [14]:
fig, ax = plot(results_simtime, do_intercept=True)      

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

                            OLS Regression Results                            
Dep. Variable:           buildCPUTime   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.070
Method:                 Least Squares   F-statistic:                   0.08426
Date:                Mon, 07 Dec 2020   Prob (F-statistic):              0.776
Time:                        09:16:28   Log-Likelihood:                -3.8948
No. Observations:                  15   AIC:                             11.79
Df Residuals:                      13   BIC:                             13.21
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.0068      0.216     23.213      0.0

/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))


In [15]:
fig.savefig(tag + '_simtime.png')

# scanning over two parameters

## Setting-up model and initializing parameters

In [16]:
tag = '2020-12-04_scan-2D_' + simulator

## Running Model and Loading Results

In [17]:
variables = ['N_pop', 'simtime']
filename = 'outputs/' + tag
for variable in variables: filename +=  '_' + variable
filename += '.json'

In [18]:
import os
import time
import pandas as pd
verbose = True
def scan(variables, values, tag=tag, verbose=verbose):
    filename = 'outputs/' + tag
    for variable in variables: filename +=  '_' + variable
    filename += '.json'
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, values_ in enumerate(values):
            if verbose: print('{variable}---{values_}'.format(values_=values_, variable=variable))
            args = {}
            for variable, value in zip(variables, values_):
                args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= dict(zip(variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'], 
                                list(values_) + [buildCPUTime, simCPUTime, writeCPUTime]))

        df.to_json(filename)
        
    return df

## Running time as a function of neuron numbers and simtime

In [19]:
import numpy as np

In [20]:
N_trials = 200
N_pops = [int(k)**2 for k in np.linspace(15, 60, N_trials, endpoint=True)]
simtimes = np.linspace(200, 3000, N_trials, endpoint=True)

In [21]:
np.random.seed(42)
np.random.shuffle(N_pops)
np.random.shuffle(simtimes)
values = zip(N_pops, simtimes)

In [22]:
results = scan(variables=variables, values=values)
results

Loaded file


,N_pop,simtime,buildCPUTime,simCPUTime,writeCPUTime
0,1296,2676.381910,4.710913,86330.511570,586.471796
1,324,664.321608,4.064798,62028.760672,116.994858
2,441,1902.512563,5.387783,74981.198549,128.357410
3,2500,2732.663317,4.662275,90571.746588,2382.756233
4,1849,1635.175879,5.050659,74322.613001,1120.280027
...,...,...,...,...,...
195,1444,2606.030151,4.149437,83879.153252,382.340908
196,324,931.658291,4.332304,64301.814318,110.289812
197,1225,2015.075377,4.499912,78651.786804,318.945408
198,3025,1958.793970,4.953384,79250.867367,2133.352995


## analysis

In [23]:
results_ext = results.copy()
results_ext['simtime_N_pop'] = results_ext['simtime'] * results_ext['N_pop']
results_ext

,N_pop,simtime,buildCPUTime,simCPUTime,writeCPUTime,simtime_N_pop
0,1296,2676.381910,4.710913,86330.511570,586.471796,3.468591e+06
1,324,664.321608,4.064798,62028.760672,116.994858,2.152402e+05
2,441,1902.512563,5.387783,74981.198549,128.357410,8.390080e+05
3,2500,2732.663317,4.662275,90571.746588,2382.756233,6.831658e+06
4,1849,1635.175879,5.050659,74322.613001,1120.280027,3.023440e+06
...,...,...,...,...,...,...
195,1444,2606.030151,4.149437,83879.153252,382.340908,3.763108e+06
196,324,931.658291,4.332304,64301.814318,110.289812,3.018573e+05
197,1225,2015.075377,4.499912,78651.786804,318.945408,2.468467e+06
198,3025,1958.793970,4.953384,79250.867367,2133.352995,5.925352e+06


In [24]:
variables_ext = ['N_pop', 'simtime_N_pop']
X = results_ext[variables_ext].values.reshape(-1, len(variables_ext))


https://aegis4048.github.io/mutiple_linear_regression_and_visualization_in_python

See https://patsy.readthedocs.io/en/latest/quickstart.html

In [25]:
import patsy
import statsmodels.api as sm

for regressor in ['buildCPUTime', 'simCPUTime', 'writeCPUTime']:
    y = results_ext[regressor].values
    #print(variables_ext, X.shape, y.shape)
    y, X = patsy.dmatrices('{regressor} ~ 0 + N_pop + simtime_N_pop'.format(regressor=regressor), data=results_ext, return_type='dataframe')

    fit = sm.OLS(y, X).fit()
    fit.summary()

    print('{regressor} (ms) = {fit0:.3f} * N_pop + {fit1:.3f}/1000 * simtime (ms) * N_pop'.format(regressor=regressor, fit0=fit.params[0], fit1=fit.params[1]*1000))

buildCPUTime (ms) = 0.002 * N_pop + 0.000/1000 * simtime (ms) * N_pop
simCPUTime (ms) = 24.476 * N_pop + 6.483/1000 * simtime (ms) * N_pop
writeCPUTime (ms) = 0.253 * N_pop + 0.186/1000 * simtime (ms) * N_pop
